# 匹配数据


* 导入整理后的数据
* 进行切词测试
* 匹配数据
    * 行政级别定位  县级
    * 按地址匹配   初步匹配
    * 容积率，面积，精确匹配
    * 地理坐标位置转换进行匹配


In [ ]:
## version python 3 
import pandas as pd
import numpy  as np
import jieba 

In [ ]:
path = "F:\\DATAbase\\land\\HDF\\"
fm_df = pd.read_excel(path+'fm_df.xlsx')
sm_df = pd.read_excel(path+'sm_df.xlsx')

In [ ]:
### 
# test for jieba 切词 
###
# add dictionary
keyword=[]
datafile=open('F:\\DATAbase\\land\\tttt3.txt')
for line in datafile.readlines():
    keyword.append(line)
    
datafile.close()


demo=list(jieba.cut((u"朝阳区幸福二村24号楼")))

def filter_address(demo):
    new2_str=[u'镇',u'路',u'村',u'楼']
    #倒叙删除
    for i in range(len(demo)-1,-1,-1):
        if  any( str_ in demo[i] for str_ in new2_str) and len(demo[i])<3 :
#             print('true')
            demo[i-1]=demo[i-1]+demo[i]
            del demo[i]
    return demo
xxx=filter_address(demo)
print(xxx)


In [ ]:
match_column=['mindex','location','address','lyear','area','price','plot_ratio']
match_column2=['mindex','level','mkey_1','location','address','lyear','area','price','plot_ratio']
cand_col_fm=['mindex',u'行政区',u'项目位置',u'土地使用年限',u'面积(公顷)',u'成交价格(万元)',u'约定容积率上限']
cand_col_fm2=['mindex','level','mkey_1',u'行政区',u'项目位置',u'土地使用年限',u'面积(公顷)',u'成交价格(万元)',u'约定容积率上限']
cand_col_sm=['mindex',u'县',u'地址',u'剩余年限',u'面积',u'转让费',u'容积率']
cand_col_sm2=['mindex','level','mkey_1',u'县',u'地址',u'剩余年限',u'面积',u'转让费',u'容积率']

fm_df.rename(columns=dict(zip(cand_col_fm,match_column)), inplace=True)
sm_df.rename(columns=dict(zip(cand_col_sm,match_column)), inplace=True)

## construct the match data 二级市场
match_sm_data=pd.DataFrame(columns = match_column2) # output matched pair data of second market
## construct the match data 一级市场
match_fm_data=pd.DataFrame(columns = match_column2 )

print(match_sm_data)
print(match_sm_data)

In [ ]:
print(fm_df.columns)
print(sm_df.columns)

In [ ]:
# #分组 一级市场
group_fm = fm_df.groupby(u'location')
county_fm_dict = dict(list(group_fm))
county_fm_name=list(county_fm_dict.keys())

# #分组 二级市场
group_sm=sm_df.groupby(u'location')
county_sm_dict = dict(list(group_sm))
county_sm_name=list(county_sm_dict.keys())

In [ ]:
result_fm_data=pd.DataFrame(columns=match_column2)
result_sm_data=pd.DataFrame(columns=match_column2)

for key in county_sm_dict:
    if key in county_fm_dict:
        temp_fm = county_fm_dict[key]
        temp_fm = temp_fm[match_column]
        temp_fm['level']=0
        temp_fm['mkey_1']=u""


        for j in range(len(county_sm_dict[key])):
            temp_sm=county_sm_dict[key].iloc[j]
            temp_sm=temp_sm[match_column]

            temp_sm['mkey_1']=""
            temp_sm['level']=0
            
            print(temp_sm['address'])
            temp_add=list(jieba.cut(temp_sm[u'address']))
            
            temp_add2=filter_address(temp_add) # get the candidated address

            # get the first market data-slice

            ## 标记等级 一级市场
            for ele in temp_add2: #循环过滤
                temp_fm.loc[temp_fm['address'].str.contains(ele)==True,'level']+=1

            ## get the maxed 
            max_match_level=np.max(temp_fm['level'])


            ## 标记等级 二级市场
            temp_fm['level'].fillna(0,inplace=True)
            temp_sm['level']=max_match_level


            ### 记录位置


            temp_fm.loc[temp_fm['level']>=1,'mkey_1']+=temp_fm['level'].astype(str)+"|"+temp_sm['mindex'].astype(str)+u"-"
#             print temp_fm['mkey_1']


            for ele in temp_fm.loc[temp_fm['level']==np.max(temp_fm['level']),'mindex']:
                temp_sm.loc['mkey_1']+=temp_sm['level'].astype(str)+"|"+ele.astype(str)+u"-"    



            ### 存储二级市场信息
            if temp_sm['level']>=1 :
                match_sm_data=match_sm_data.append(temp_sm,ignore_index=True)

        ### rename the dataframe    
#         temp_fm.rename(columns=dict(zip(cand_col_fm2,match_column)), inplace=True)
#         result_sm_data.rename(columns=dict(zip(cand_col_sm2,match_column)), inplace=True)

        ### save the result
        temp_fm=temp_fm[temp_fm['level']>=1]
        match_fm_data=match_fm_data.append(temp_fm,ignore_index=True)
        
        result_sm_data=[]
         

In [ ]:
print(match_sm_data.shape)
print(match_fm_data.shape)

In [ ]:
match_fm_data.to_excel(path+'match_fm.xlsx','sheet1')
match_sm_data.to_excel(path+'match_sm.xlsx','sheet1')